# Install, Import, and Log In

In [1]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

### Step 0: Install W&B

In [2]:
%%capture
!pip install wandb

### Step 1: Import W&B and Login

In [12]:
import wandb
from wandb.keras import WandbCallback

project_name = 'project-fashion-mnist'
# %env WANDB_API_KEY='1af04e33e3d441eb82eb612e2c001eddec29bccb'

wandb.login(relogin=True)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Avni Eshchar/.netrc


True

> Side note: If this is your first time using W&B or you are not logged in, the link that appears after running `wandb.login` will take you to sign-up/login page. Signing up is easy!

# Download and Prepare the Dataset


In [4]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Subsetting train data and normalizing to [0., 1.]
x_train, x_test = x_train / 255., x_test / 255.

class_names = ['T_shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

input_shape = (28, 28, 1)
num_classes = len(class_names)

print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_test: ', y_test.shape)
print('Num of classes: ', num_classes)

Shape of x_train:  (60000, 28, 28)
Shape of y_train:  (60000,)
Shape of x_test:  (10000, 28, 28)
Shape of y_test:  (10000,)
Num of classes:  10


# Define the Model

Here, we define a standard CNN (with convolution and max-pooling) in Keras.

In [5]:
def Model():
    inputs = keras.layers.Input(shape=input_shape)

    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(inputs)
    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.GlobalAveragePooling2D()(x)

    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dense(28, activation='relu')(x)

    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)

# Train the Model

In [13]:
# Initialize wandb with your project name
run = wandb.init(project=project_name,
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "fashion_mnist"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize model like you usually do.
tf.keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do.
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
model.compile(optimizer, config.loss_function, metrics=['acc'])

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


In [7]:
# We focus on a subset of images, since this is for human review
val_images, val_labels = x_test[:32], y_test[:32]

_ = model.fit(x_train, y_train,
              epochs=config.epochs, 
              batch_size=config.batch_size,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback(data_type='image', 
                                       validation_data=(val_images, val_labels), 
                                       labels=class_names)])

run.finish()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Epoch 1/5
2/2 [==============================] - 0s 4ms/step0s - loss: 0.8294 - acc: 0.68


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230101_141438-2cfx9nfp\files\model-best\assets


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230101_141438-2cfx9nfp\files\model-best\assets
wandb: Adding directory to artifact (d:\dev\elbit\ML-course\project-2\wandb\run-20230101_141438-2cfx9nfp\files\model-best)... Done. 0.0s


1875/1875 [==============================] - 45s 23ms/step - loss: 0.8296 - acc: 0.6842 - val_loss: 0.6585 - val_acc: 0.7487
Epoch 2/5
2/2 [==============================] - 0s 3ms/step0s - loss: 0.5412 - acc: 0.79


...

1875/1875 [==============================] - 44s 23ms/step - loss: 0.3825 - acc: 0.8605 - val_loss: 0.3777 - val_acc: 0.8607
Epoch 5/5
2/2 [==============================] - 0s 3ms/step0s - loss: 0.3439 - acc: 0.87


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230101_141438-2cfx9nfp\files\model-best\assets


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230101_141438-2cfx9nfp\files\model-best\assets
wandb: Adding directory to artifact (d:\dev\elbit\ML-course\project-2\wandb\run-20230101_141438-2cfx9nfp\files\model-best)... Done. 0.0s


1875/1875 [==============================] - 44s 23ms/step - loss: 0.3439 - acc: 0.8738 - val_loss: 0.3656 - val_acc: 0.8746


acc,▁▅▇██
epoch,▁▃▅▆█
loss,█▄▂▂▁
val_acc,▁▅▇▇█
val_loss,█▄▂▁▁
acc,0.87383
best_epoch,4
best_val_loss,0.36555
epoch,4
loss,0.34386
val_acc,0.8746


# Test Hyperparameters with Sweeps

In [14]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project=project_name)
wandb.agent(f'eshcharc/{project_name}/{sweep_id}')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Exception: function paramter must be callable!